# Project 2 - File for Project 1 Models

Data from: Heyes, Anthony, and Soodeh Saberian. 2019. "Temperature and Decisions: Evidence from 207,000 Court Cases." American Economic Journal: Applied Economics, 11 (2): 238–65.

Notebooks used troughout the code: 
- ISLP-Ch06_varselect_lab.ipynb
- ISLP-TreeModels.ipynb
- CIDP-Chapter_04
- CIDP-Chapter_05
- CIDP-Chapter_07
- CIBT-11-Propensity-Score

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as skm
from sklearn.model_selection import GridSearchCV, KFold
from matplotlib.pyplot import subplots
from statsmodels.discrete.discrete_model import Probit
import statsmodels.api as sm
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier

!pip install stargazer
from stargazer.stargazer import Stargazer

In [ ]:
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestClassifier as RFC,
      GradientBoostingClassifier as GBC)
from sklearn.metrics import confusion_matrix

In [ ]:
from scipy import stats

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import graphviz
import networkx as nx
COLORS = [
    '#00B0F0',
    '#FF0000'
]
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter

In [ ]:
!pip install dowhy
import dowhy
from dowhy import CausalModel

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.linear_model import LassoCV
!pip install econml
from econml.dr import LinearDRLearner
from joblib import Parallel, delayed 
import shap
from sklearn.preprocessing import StandardScaler
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

## Data Description 

In [ ]:
df = pd.read_stata('matched_corrected.dta')
df.describe()

In [ ]:
#Create a dummy for asylum
df['dummy_asylum'] = df['c_asy_type'].apply(lambda x: 1 if x == 'E' else 0)
#Create a dummy for gender
df['dummy_gender'] = df['gender'].apply(lambda x: 1 if x == 'female' else 0)

In [ ]:
#As outlined in the correction article drop the observation for China
df = df[df['nat_name'] != 'CHINA']

In [ ]:
# Get unique values to identify variables for the dummy variables
unique__names = df['nat_name'].unique()
locations = df['location'].unique()

In [ ]:
# Define the list of regions
middle_eastern_countries = ["BAHRAIN", "CYPRUS", "EGYPT", "IRAN", "IRAQ", "ISRAEL", "JORDAN", 
    "KUWAIT", "LEBANON", "OMAN", "PALESTINE", "QATAR", "SAUDI ARABIA", 
    "SYRIA", "TURKEY", "UNITED ARAB EMIRATES", "YEMEN"]

africa = ["ERITREA", "RWANDA", "SOMALIA", "SUDAN", "CONGO", "ETHIOPIA", "LIBYA", 
    "MALI", "ANGOLA", "BURUNDI", "TANZANIA", "NIGERIA", "GABON", "GHANA", 
    "SENEGAL", "CHAD", "DJIBOUTI", "CAMEROON", "UGANDA", "KENYA", 
    "ZAMBIA", "MAURITANIA", "SOUTH AFRICA", "GUINEA", "BURKINA FASO", 
    "MOROCCO", "ALGERIA", "COMORO ISLANDS", "EQUATORIAL GUINEA", 
    "CENTRAL AFRICAN REPUBLIC", "CAPE VERDE", "LESOTHO", "SWAZILAND", 
    "GAMBIA", "SIERRA LEONE", "GUINEA BISSAU"]

america = ["GUATEMALA", "EL SALVADOR", "PANAMA", "COLOMBIA", 
    "ARGENTINA", "HAITI", "VENEZUELA", "MEXICO", "CUBA", "DOMINICAN REPUBLIC", 
    "BRAZIL", "CHILE", "SURINAME", "TRINIDAD AND TOBAGO", "JAMAICA", 
    "CANADA", "USA", "ST. KITTS, WEST INDIES", "ANTIGUA AND BARBUDA", 
    "BARBADOS", "BAHAMAS", "BELIZE", "DOMINICA", "GRENADA", 
    "NICARAGUA", "URUGUAY", "PARAGUAY", "ST. LUCIA", "ST. VINCENT AND THE GRENADINES"]

asia = ["PAKISTAN", "VIETNAM", "INDONESIA", "AFGHANISTAN", 
    "IRAN", "BANGLADESH", "PHILIPPINES", "TAIWAN", "MALAYSIA", 
    "KAZAKHSTAN", "KYRGYZSTAN", "THAILAND", "TURKMENISTAN", "UZBEKISTAN", 
    "MONGOLIA", "SRI LANKA", "BHUTAN", "LAOS", "NEPAL", 
    "MYANMAR", "KAMPUCHEA", "BRUNEI", "BURMA", "KOREA", "NORTH KOREA"]

europe = ["RUSSIA", "ARMENIA", "ALBANIA", "YUGOSLAVIA", "UNITED KINGDOM", 
    "BULGARIA", "ROMANIA", "HUNGARY", "POLAND", "CZECH REPUBLIC", 
    "SLOVAK REPUBLIC", "GERMANY", "FRANCE", "ITALY", "SPAIN", 
    "SWEDEN", "DENMARK", "FINLAND", "AUSTRIA", "SWITZERLAND", 
    "BELGIUM", "GREECE", "NETHERLANDS", "CROATIA", "SLOVENIA", 
    "MONACO", "LITHUANIA", "LATVIA", "ESTONIA", "ICELAND"]

df['middleast'] = 0
df['america'] = 0
df['africa'] = 0
df['asia'] = 0
df['europe'] = 0

df.loc[df['nat_name'].isin(middle_eastern_countries), 'middleast'] = 1
df.loc[df['nat_name'].isin(america), 'america'] = 1
df.loc[df['nat_name'].isin(africa), 'africa'] = 1
df.loc[df['nat_name'].isin(asia), 'asia'] = 1
df.loc[df['nat_name'].isin(europe), 'europe'] = 1

#Create interaction terms
df['middleast_dev'] = df['middleast']*df['temp6t4']
df['america_dev'] = df['america']*df['temp6t4']
df['africa_dev'] = df['africa']*df['temp6t4']
df['asia_dev'] = df['asia']*df['temp6t4']
df['europe_dev'] = df['europe']*df['temp6t4']

In [ ]:
#Define list of locations
northeast = ['NEWARK', 'BOSTON', 'NEW YORK CITY', 'BUFFALO', 'PHILADELPHIA', 
    'NEW YORK ANNEX', 'NY DET (VARICK ST.)', 'HARTFORD', 
    '*PA DOC.', 'CLEVELAND', '*BOP  DANBURY', '*RI  DOC',
    '*WISCONSIN DOC', '*NH  DOC', '*SUFFOLK COUNTY','*NEWARK VIDEO HEARINGS','*JESSUP'
    '*BOP ALLENWOOD', '*NORTHERN STATE NJ DOC','YORK COUNTY DET','YORK COUNTY DET']

midwest = ['CHICAGO', 'DETROIT', 'CINCINNATI', 'CLEVELAND', 'ST. LOUIS', 
    'MEMPHIS', 'KANSAS CITY', 'OMAHA', '*MI  DOC', 
    '*IL DOC - STATESVILLE', '*MO DOC', '*OHIO DOC', 
    '*INDIANA YOUTH CENTER']

south = ['ARLINGTON', 'DALLAS', 'HOUSTON', 'MIAMI', 'ATLANTA', 
    'NEW ORLEANS', 'SAN ANTONIO', 'DALLAS DET', 'SAN ANTONIO DET', 
    'HOUSTON DET', 'ATLANTA DET', '*GEORGIA DOC', '*VA DOC', 
    '*DADE COUNTY FL DOC', '*BROWARD  FL DOC', 'ORLANDO', 'KROME DET',
    'PORT ISABEL DET', 'EL PASO', 'EL PASO DET', '*TX DOC', 
    'LOUISVILLE', 'OKLAHOMA CITY', 'OKLAHOMA CITY DET', 
    'BATAVIA SPC', 'BROWARD TRANS CTR','ST. THOMAS', 'ST. CROIX', 'ROLLING PLAINS DETENTION CENTER',
    '*BOP BIG SPRING AIRPARK','BRADENTON DET','SAN ANTONIO DET']

west = ['DENVER', 'SAN DIEGO', 'LOS ANGELES', 'SAN FRANCISCO', 
    'PHOENIX', 'LAS VEGAS', 'RENO', 'SALT LAKE CITY', 'OTAY MESA', 
    'TUCSON', 'HONOLULU', 'SAN JUAN', 'SEATTLE', 'PORTLAND',
    'SAN FRANCISCO DET', 'DENVER DET', 'SAN DIEGO DETAINED', 
    'MIRA LOMA DET', 'HONOLULU DET', '*CO DOC', '*AZ DOC',
    '*WA DOC', '*AK DOC', 'ANCHORAGE', 'SAN PEDRO', 
    'IMPERIAL', '*NM DOC','PORTLAND DET','*MONROE WA DOC','SAN FRANCISCO ANNEX']


df['northeast'] = 0
df['midwest'] = 0
df['south'] = 0
df['west'] = 0

df.loc[df['location'].isin(northeast), 'northeast'] = 1
df.loc[df['location'].isin(midwest), 'midwest'] = 1
df.loc[df['location'].isin(south), 'south'] = 1
df.loc[df['location'].isin(west), 'west'] = 1

#Create interaction terms
df['middleast_dev'] = df['middleast']*df['temp6t4']
df['america_dev'] = df['america']*df['temp6t4']
df['africa_dev'] = df['africa']*df['temp6t4']
df['asia_dev'] = df['asia']*df['temp6t4']
df['europe_dev'] = df['europe']*df['temp6t4']

locations = ['northeast', 'midwest', 'south', 'west']
nationalities = ['middleast', 'america', 'africa', 'asia', 'europe']

for nationality in nationalities:
    for location in locations:
        interaction_column_name = f'{location}_{nationality}_dev'
        df[interaction_column_name] = df[location] * df[nationality] * df['temp6t4']
        
for location in locations:
    interaction_column_name = f'{location}_dev'
    df[interaction_column_name] = df[location] * df['temp6t4']

In [ ]:
# Create dummy variables for the months 
df['month'] = df['date'].dt.month
df = pd.get_dummies(df, columns=['month'], prefix='month', drop_first=False)

#Dummy for winter and summer
df['month'] = df['date'].dt.month

df['summer_spring'] = df['month'].isin([3, 4, 5, 6, 7, 8]).astype(int)
df['winter_fall'] = df['month'].isin([1, 2, 9, 10, 11, 12]).astype(int)

seasons = ['summer_spring', 'winter_fall']

In [ ]:
#Clean the dataset
df = df[df['c_asy_type'].isin(['E', 'I'])]
df_final = df.dropna(axis=0) 

## Summary Statistics

In [ ]:
summary_stats = df_final[['temp6t4','heat','wind6t4','skycover','ozone','pm']].describe() 
print(summary_stats)

In [ ]:
range_temp = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  
labels = [f"{range_temp[i]}-{range_temp[i+1]}" for i in range(len(range_temp) - 1)]  

df_final['temp_bins'] = pd.cut(df_final['temp6t4'], bins=range_temp, labels=labels, right=False)

counts = df_final['temp_bins'].value_counts().sort_index()
percentages = counts / counts.sum() * 100

plt.figure(figsize=(7, 6), facecolor='white')  
ax = plt.subplot(111, facecolor='white')  
percentages.plot(kind='bar', color='skyblue')
plt.xlabel('Temperature (°F)')
plt.ylabel('Percentage of Cases')
plt.xticks(rotation=45)
plt.grid(axis='y')

ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.0f}%'))
for spine in ax.spines.values():
    spine.set_visible(False)
    
plt.tight_layout()
plt.show()

## OLS

In [ ]:
Y = np.array(df_final['res'])

In [ ]:
#Create X variables for different specifications
#Note: drop one category for each dummy

#Specification 1
selectedvariables = ['temp6t4','heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

#Specification 2
selectedvariables_noweather = ['temp6t4','chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1','month_2',
                      'month_3','month_4','month_5','month_6','month_7','month_8',
                    'month_9','month_10','month_11']

# Deviation Specification 
selectedvariables_deviation = ['deviation','heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']
                        
#Create X variables with different specification
X = df_final[selectedvariables]

X_no_control = df_final[selectedvariables_noweather]

X_deviation = df_final[selectedvariables_deviation]

In [ ]:
#Specification 1
model_1 = Probit(Y, X.astype(float))
probit_model1 = model_1.fit()
print(probit_model1.summary())

In [ ]:
# Calculate marginal effect for the variable of interest
predicted_probs = probit_model1.predict(X.astype(float))

x_temp6t4 = X['temp6t4']  
marginal_effect_temp = probit_model1.params['temp6t4'] * predicted_probs * (1 - predicted_probs)
average_marginal_effect_temp = np.mean(marginal_effect_temp)

print(average_marginal_effect_temp)

In [ ]:
stargazer = Stargazer([probit_model1])
print(stargazer.render_latex())

In [ ]:
#Specification 1 - Deviation 

model_1_deviation = Probit(Y, X_deviation.astype(float))
probit_model1_deviation = model_1_deviation.fit()
print(probit_model1_deviation.summary())

In [ ]:
stargazer_dev = Stargazer([probit_model1_deviation])
print(stargazer_dev.render_latex())

In [ ]:
#Specification 2
model_2 = Probit(Y, X_no_control.astype(float))
probit_model2 = model_2.fit()
print(probit_model2.summary())

In [ ]:
stargazer_2 = Stargazer([probit_model2])
print(stargazer_2.render_latex())

In [ ]:
# Calculate marginal effect for the variable of interest 
predicted_probs = probit_model2.predict(X_no_control.astype(float))
x_temp6t4 = X['temp6t4']  
marginal_effect_temp = probit_model2.params['temp6t4'] * predicted_probs * (1 - predicted_probs)
average_marginal_effect_temp = np.mean(marginal_effect_temp)

print(average_marginal_effect_temp)

In [ ]:
# Filter per month 
df_final['date'] = pd.to_datetime(df_final['date'])

#Filter the dataset as winter/fall and summer/spring
df_filter_nowinter = df_final[df_final['date'].dt.month.isin([3, 4, 5, 6, 7, 8])]
df_filter_winter = df_final[df_final['date'].dt.month.isin([1,2,9,10,11,12])] 

In [ ]:
# Calculate the average and standard deviation of resolution winter and non-winter months
winter_mean = df_filter_winter['res'].mean()
winter_std = df_filter_winter['res'].std()
nowinter_mean = df_filter_nowinter['res'].mean()
nowinter_std = df_filter_nowinter['res'].std()

summary = pd.DataFrame({
    'season': ['Winter and Fall', 'Summer and Spring'],
    'mean': [winter_mean, nowinter_mean],
    'std_dev': [winter_std, nowinter_std]
})

plt.figure(figsize=(7, 7), facecolor='white')
ax = plt.subplot(111, facecolor='white')
ax.grid(False) 

bars = plt.bar(summary['season'], summary['mean'], yerr=summary['std_dev'], capsize=5, color=['skyblue', 'lightcoral'])
plt.ylabel('Average Resolution')
for spine in ax.spines.values():
    spine.set_visible(False)

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the average and standard deviation of deviation of average temp for winter and non-winter months
winter_mean_temp = df_filter_winter['deviation'].mean()
winter_std_temp = df_filter_winter['deviation'].std()
nowinter_mean_temp = df_filter_nowinter['deviation'].mean()
nowinter_std_temp = df_filter_nowinter['deviation'].std()

summary_temp = pd.DataFrame({
    'season': ['Winter and Fall', 'Summer and Spring'],
    'mean': [winter_mean_temp, nowinter_mean_temp],
    'std_dev': [winter_std_temp, nowinter_std_temp]
})

plt.figure(figsize=(7, 7), facecolor='white')
ax = plt.subplot(111, facecolor='white')
ax.grid(False) 

bars = plt.bar(summary_temp['season'], summary_temp['mean'], yerr=summary_temp['std_dev'], capsize=5, color=['skyblue', 'lightcoral'])
plt.ylabel('Average Deviation')
for spine in ax.spines.values():
    spine.set_visible(False)

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## OLS for Project 2

In [ ]:
categorical_vars = ['chair']
df_dummies = pd.get_dummies(df_final, columns=categorical_vars, drop_first=True)

#Drop one category for each dummy
drop_columns = ['month','summer_spring','west_asia_dev','west_dev','asia','west','month_12','asia_dev','date','location','city', 'id1','id','temp0t7','gender','nat_name','c_asy_type','temp_daily', 'press6t4', 'press0t7', 'press_daily', 'dew6t4', 'dew0t7', 'dew_daily', 'prcp6t4', 'prcp0t7', 'prcp_daily', 'wind6t4', 'wind0t7', 'wind_daily', 'rh6t4', 'deviation', 'ltemp6t4', 'l2temp6t4', 'l3temp6t4', 'letemp6t4', 'le2temp6t4', 'le3temp6t4', 'beforetemp', 'aftertemp', 'temp22t8', 'press22t8', 'prcp22t8', 'wind22t8', 'dew22t8', 'temp4t4', 'press4t4', 'dew4t4', 'prcp4t4', 'wind4t4', 'lprcp', 'heat', 'comp_date', 'hearing_loc_code']
df_dummies_clean = df_dummies.drop(drop_columns, axis=1)

In [ ]:
Y_v2 = df_dummies_clean['res']
X_base = df_dummies_clean.drop(columns=['res'])
X_v2 = sm.add_constant(X_base)

In [ ]:
model_v2 = Probit(Y, X.astype(float))  
probit_model_v2 = model_v2.fit()
print(probit_model_v2.summary())

In [ ]:
stargazer_v2 = Stargazer([probit_model_v2])
print(stargazer_v2.render_latex())

## Running Ridge 

In [ ]:
#Code in this section based on the notebook: ISLP-Ch06_varselect_lab.ipynb

coefficients = []
lambdas = 10**np.linspace(8, -2, 100) / Y_v2.std()
scaler = StandardScaler(with_mean=True,  with_std=True)
for lam in lambdas:
    ridge = SGDClassifier(loss='log_loss', penalty='l2',alpha=lam)
    pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
    pipe.fit(X_v2, Y_v2)
    coefficients.append(pipe.named_steps['ridge'].coef_.flatten())

soln_array = np.array(coefficients)

soln_path = pd.DataFrame(soln_array, columns=X_v2.columns, index=-np.log(lambdas))
soln_path.index.name = 'negative log(lambda)'

In [ ]:
path_fig, ax = subplots(figsize=(8,8))
soln_path.plot(ax=ax, legend=False)
ax.set_xlabel('$-\log(\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
ax.legend(loc='upper left');

path_fig.patch.set_facecolor('white')  
ax.set_facecolor('white')

In [ ]:
kfold = KFold(n_splits=5)

param_grid = {'ridge__alpha': lambdas}
grid_search = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy', return_train_score=True)
grid_search.fit(X_v2, Y_v2)
tuned_ridge = grid_search.best_estimator_.named_steps['ridge']

mean_scores = grid_search.cv_results_['mean_test_score']
std_scores = grid_search.cv_results_['std_test_score']

In [ ]:
ridgeCV_fig, ax = subplots(figsize=(8, 8))
ax.errorbar(-np.log(lambdas), mean_scores, yerr=std_scores / np.sqrt(kfold.get_n_splits()), fmt='o')
ax.axvline(-np.log(grid_search.best_params_['ridge__alpha']), c='k', ls='--')
ax.set_xlabel('$-\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
ax.set_title('Cross-Validation Accuracy with Error Bars')

In [ ]:
coefficients = tuned_ridge.coef_.flatten()  
variable_names = X_v2.columns  

coef_mapping = {variable: coef for variable, coef in zip(variable_names, coefficients)}
coef_df = pd.DataFrame(list(coef_mapping.items()), columns=['Variable', 'Coefficient'])
print(coef_df)

In [ ]:
pd.set_option('display.max_rows', 306)  
print(coef_df)

## Running Lasso

In [ ]:
#Code in this section based on the notebook: ISLP-Ch06_varselect_lab.ipynb

coefficients_l = []
lambdas = 10**np.linspace(8, -2, 100) / Y.std()
scaler = StandardScaler(with_mean=True,  with_std=True)
for lam in lambdas:
    lasso = SGDClassifier(loss='log_loss', penalty='l1',alpha=lam)
    pipe_l = Pipeline(steps=[('scaler', scaler), ('lasso', lasso)])
    pipe_l.fit(X_v2, Y_v2)
    coefficients_l.append(pipe_l.named_steps['lasso'].coef_.flatten())

soln_array_l = np.array(coefficients_l)
soln_path_l = pd.DataFrame(soln_array_l, columns=X.columns, index=-np.log(lambdas))
soln_path_l.index.name = 'negative log(lambda)'

In [ ]:
path_fig_l, ax = subplots(figsize=(8,8))
soln_path_l.plot(ax=ax, legend=False)
ax.set_xlabel('$-\log(\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
ax.legend(loc='upper left');

In [ ]:
kfold = KFold(n_splits=5)

param_grid_l = {'lasso__alpha': lambdas}


grid_search_l = GridSearchCV(pipe_l, param_grid_l, cv=kfold, scoring='accuracy', return_train_score=True)
grid_search_l.fit(X, Y)
tuned_lasso = grid_search_l.best_estimator_.named_steps['lasso']

mean_scores_l = grid_search_l.cv_results_['mean_test_score']
std_scores_l = grid_search_l.cv_results_['std_test_score']

In [ ]:
lassoCV_fig, ax = subplots(figsize=(8, 8))
ax.errorbar(-np.log(lambdas), mean_scores_l, yerr=std_scores_l / np.sqrt(kfold.get_n_splits()), fmt='o')
ax.axvline(-np.log(grid_search_l.best_params_['lasso__alpha']), c='k', ls='--')
ax.set_xlabel('$-\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
ax.set_title('Cross-Validation Accuracy with Error Bars')

In [ ]:
coefficients_l = tuned_lasso.coef_.flatten()  
variable_names = X_v2.columns  

coef_mapping_l = {variable: coef for variable, coef in zip(variable_names, coefficients_l)}

coef_df = pd.DataFrame(list(coef_mapping_l.items()), columns=['Variable', 'Coefficient'])

In [ ]:
pd.set_option('display.max_rows', 306)  
sorted_df = coef_df.sort_values(by='Coefficient', ascending=False)
print(sorted_df)

## Regresion Trees 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

clf = DTC(criterion='entropy', 
          max_depth = 3,
          random_state=0)   

clf.fit(X, Y)

In [ ]:
X.columns
#Rename columns for better labelling of trees
X_detailed = ['Average temperature', 'Heat index', 'Sky coverage', 'Carbon monoxide levels', 
              'Distance CO source', 'Ozone levels', 'Distance ozone source', 'PM levels', 
              'Distance PM source', 'Atmospheric pressure', 'Dew point temperature', 'Precipitation', 
              'Wind speed', 'Relative humidity', 'Judge identifier', 'Asylum application', 
              'Gender', 'Middle Eastern', 'American', 'African', 'European', 'Northeast', 'Midwest', 'South', 
              '2000','2001', '2002', '2003', 'Interaction of temperature and Middle Eastern',
              'Interaction of temperature and American', 'Interaction of temperature and African', 
              'Interaction of temperature and European', 'January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November']

In [ ]:
feature_names = X_detailed

In [ ]:
accuracy_score(Y, clf.predict(X)) 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

fig.patch.set_facecolor('white')  
ax.set_facecolor('white')        

plot_tree(clf,
          feature_names=feature_names,
          ax=ax,
          filled=True,  
          rounded=True,  
          fontsize=7,   
          proportion=True)  

plt.show()

In [ ]:
#Cross validation 
validation = skm.ShuffleSplit(n_splits=1,
                              test_size=200,
                              random_state=0)
results = skm.cross_validate(clf,
                             X,
                             Y,
                             cv=validation)
results['test_score']

In [ ]:
#Split dataset
(X_train,
 X_test,
 Y_train,
 Y_test) = skm.train_test_split(X,
                                   Y,
                                   test_size=0.5,
                                   random_state=0)

In [ ]:
clf = DTC(criterion='entropy', random_state=0)
clf.fit(X_train, Y_train)
accuracy_score(Y_test, clf.predict(X_test))

In [ ]:
ccp_path = clf.cost_complexity_pruning_path(X_train, Y_train)
kfold = skm.KFold(5,
                  random_state=1,
                  shuffle=True)

## Bagging 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

bag_temperature = RFC(max_features=X_train.shape[1], random_state=0)
bag_temperature.fit(X_train, Y_train)

In [ ]:
bag_temperature = RFC(max_features=X_train.shape[1],
                n_estimators=500,
                random_state=0).fit(X_train, Y_train)
y_hat_bag = bag_temperature.predict(X_test)
accuracy_bagging = accuracy_score(Y_test, y_hat_bag)
accuracy_bagging

In [ ]:
feature_imp_bag = pd.DataFrame(
    {'importance':bag_temperature.feature_importances_},
    index=feature_names)
feature_imp_bag.sort_values(by='importance', ascending=False)

In [ ]:
feature_imp_bag = pd.DataFrame(
    {'importance': bag_temperature.feature_importances_},
    index=feature_names
)

# Sort the feature importances
feature_imp_bag = feature_imp_bag.sort_values(by='importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_imp_bag.index, feature_imp_bag['importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importances (Bagged Model)')
plt.gca().invert_yaxis()  
plt.show()

## Random Forests 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

RF_temperature = RFC(max_features=6,
               random_state=0).fit(X_train, Y_train)
y_hat_RF = RF_temperature.predict(X_test)
accuracy_RF = accuracy_score(Y_test, y_hat_RF)
accuracy_RF

In [ ]:
feature_imp = pd.DataFrame(
    {'importance':RF_temperature.feature_importances_},
    index=feature_names)
feature_imp.sort_values(by='importance', ascending=False)

## Boosting

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

boost_temperature = GBC(n_estimators=500,
                   learning_rate=0.001,
                    max_depth = 3,
                   random_state=0)
boost_temperature.fit(X_train, Y_train)

In [ ]:
boost_temperature = GBC(n_estimators=500,
                   learning_rate=0.001,
                    max_depth = 3,
                   random_state=0)
boost_temperature.fit(X_train,Y_train)
y_hat_boost = boost_temperature.predict(X_test);
accuracy_boosting = accuracy_score(Y_test, y_hat_boost)
accuracy_boosting

In [ ]:
feature_imp = pd.DataFrame(
    {'importance':boost_temperature.feature_importances_},
    index=feature_names)
feature_imp.sort_values(by='importance', ascending=False)

## Directed Acyclic Graph (DAG) and Causal Relationship 

In [ ]:
#Code in this section based on the notebook: CIDP-Chapter_04

sample_gml = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 5
    target 4
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 4
    ]
edge [
    source 5
    target 2
    ]
]
    
    """

In [ ]:
graph = nx.parse_gml(sample_gml)

nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
sample_gml2 = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 5
    target 4
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 1
    ]
edge [
    source 5
    target 2
    ]
]
    
    """

In [ ]:
graph = nx.parse_gml(sample_gml2)

nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
sample_gml3 = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 1
    ]
edge [
    source 5
    target 2
    ]
edge [
    source 5
    target 6
    ]
]
    
    """

In [ ]:
graph = nx.parse_gml(sample_gml3)

nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
gml_final = """graph [
directed 1
    
node [
    id 1
    label "midwest"
    ]

node [
    id 2
    label "deviation"
    ]

node [
    id 4
    label "res"
    ]
    
node [
    id 5
    label "northeast"
    ]

node [
    id 6
    label "chair"
    ]

node [
    id 7
    label "america"
    ]
node [
    id 8
    label "south"
    ]
node [
    id 9
    label "west"
    ]
node [
    id 11
    label "cognitive"
    ]
    
edge [
    source 2
    target 11
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 7
    target 11
    ]
edge [
    source 5
    target 2
    ]
edge [
    source 1
    target 2
    ]
edge [
    source 8
    target 2
    ]
edge [
    source 9
    target 2
    ]
edge [
    source 5
    target 11
    ]
edge [
    source 1
    target 11
    ]
edge [
    source 8
    target 11
    ]
edge [
    source 9
    target 11
    ]
edge [ 
    source 11
    target 6
    ]
]

    
    """

In [ ]:
graph = nx.parse_gml(gml_final)

nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

## Test DAG

In [ ]:
#Code in this section based on the notebook:  CIDP-Chapter_07

model = CausalModel(
data=df_final,
treatment=['deviation'],
outcome="res",
graph=gml_final)

In [ ]:
estimand = model.identify_effect()

In [ ]:
print(estimand)

In [ ]:
estimate = model.estimate_effect(
identified_estimand=estimand,
method_name="backdoor.linear_regression")

In [ ]:
print(estimate)

In [ ]:
#Refutation test on whether estimate is influenced by unobserved confounders = random_common_cause 
refute_subset = model.refute_estimate(
estimand=estimand,
estimate=estimate,
method_name="random_common_cause",
subset_fraction=0.4)

In [ ]:
print(refute_subset)
#Note: High p-value suuggests that the random common cause does not have a meaningful impact on the relationship between 
#temperature and the outcome, providing confidence in the stability of findings.

## IPW

In [ ]:
#Code in this section based on the notebook: CIBT-11-Propensity-Score

#Changed for deviation, being the treatment variable because if we had only temperature 
#we might have that specific regions such as Texas is always treated etc.
df_final['T_binary'] = (df_final['deviation'] > 0.000095).astype(int)
print(df_final['T_binary'].value_counts())

In [ ]:
T = 'T_binary'
Y = 'res'
X = ['chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

ps_model = LogisticRegression(C=1e6).fit(df_final[X], df_final[T])

data_ps = df_final.assign(propensity_score=ps_model.predict_proba(df_final[X])[:, 1])

data_ps[["T_binary", "res", "propensity_score"]].head()

In [ ]:
weight_t = 1/data_ps.query("T_binary==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("T_binary==0")["propensity_score"])
print("Original Sample Size", df.shape[0])
print("Treated Population Sample Size", sum(weight_t))
print("Untreated Population Sample Size", sum(weight_nt))

In [ ]:
sns.distplot(data_ps.query("T_binary==0")["propensity_score"], kde=False, label="Non Treated")
sns.distplot(data_ps.query("T_binary==1")["propensity_score"], kde=False, label="Treated")
plt.title("Positivity Check")
plt.legend();

In [ ]:
# Remove observations with propensity score = 1
data_ps = data_ps[data_ps["propensity_score"] < 1]

treated_data = data_ps.query("T_binary == 1")
control_data = data_ps.query("T_binary == 0")

y1 = sum(treated_data["T_binary"] * weight_t) / len(treated_data)
y0 = sum(control_data["T_binary"] * weight_nt) / len(control_data)

ate = np.mean(weight_t * treated_data["T_binary"]) - np.mean(weight_nt * control_data["T_binary"])

print("Y1:", y1)
print("Y0:", y0)
print("ATE:", ate)

In [ ]:
# Calculate weights for treated and control groups
treated_data = data_ps.query("res == 1")
control_data = data_ps.query("res == 0")

if not treated_data.empty:
    weight_t = 1 / treated_data["propensity_score"]
    print("Weight_t:", weight_t)

if not control_data.empty:
    weight_nt = 1 / (1 - control_data["propensity_score"])
    print("Weight_nt:", weight_nt)


In [ ]:
print(data_ps["propensity_score"].min(), data_ps["propensity_score"].max())


In [ ]:
def run_ps(df_final, X, T, y):
    ps = LogisticRegression(C=1e6, max_iter=2000, solver='liblinear').fit(df_final[X], df_final[T]).predict_proba(df_final[X])[:, 1]
    weight = (df_final[T]-ps) / (ps*(1-ps)) 
    return np.mean(weight * df_final[y]) 

sample_df = df_final.sample(frac=1, replace=True)
ate = run_ps(sample_df, X, T, Y)
print(ate)

In [ ]:
sample_df = df_final.sample(frac=0.1, replace=True)

In [ ]:
def run_ps(sample_df, X, T, y):
    ps = LogisticRegression(C=1e6, max_iter=2000, solver='liblinear').fit(sample_df[X], sample_df[T]).predict_proba(sample_df[X])[:, 1]
    weight = (sample_df[T]-ps) / (ps*(1-ps)) 
    return np.mean(weight * sample_df[y]) 

np.random.seed(88)
bootstrap_sample = 100
ates = Parallel(n_jobs=4)(delayed(run_ps)(sample_df.sample(frac=1, replace=True), X, T, Y)
                          for _ in range(bootstrap_sample))
ates = np.array(ates)
ates

In [ ]:
sns.distplot(ates, kde=False)
plt.vlines(np.percentile(ates, 2.5), 0, 30, linestyles="dotted")
plt.vlines(np.percentile(ates, 97.5), 0, 30, linestyles="dotted", label="95% CI")
plt.legend();